# 丢弃层

神经网络模型是怎么从图像的像素数据中学习的？

看一张宠物照片，我们可以迅速识别出拍的是猫、还是狗。如果我们把这张照片数字化，直接面对一长串的像素数值，没有谁能弄明白照片拍的是个啥！神经网络模型也无法从一张照片的数据做出判断，而是需要通过对大量相似照片的数据进行深度分析，去寻找像素之间隐藏的统计规律。

在训练过程中，网络模型里的成千上万个神经元会从不同的角度去分析每个像素点的数值和相互位置关系。通过不断迭代，逐步挑选出和答案相匹配的关系。并以此作为规则，用来对其他照片进行判断。

但是这里也有一个问题：如果照片里包括很多冗余信息，比如背景、杂物；或者过多的细节，比如动物皮毛的花色纹理等。网络模型也会一视同仁地把这些信息总结到它的规则里面。

比如：如果照片里的猫身上都有清晰的条纹，那么网络模型在分析其他照片时，会去寻找类似的数据排列，认为只有”条纹猫“才是猫。这种现象在神经网络中被称为**过拟合**（Overfitting）。

---

为了解决网络模型”学习过多细节“的问题，“深度学习三巨头”之一的辛顿（Hinton）和他的研究团队提出了一个简单、却极具智慧的机制：**丢弃层**（Dropout Layer）。

丢弃层的逻辑非常简单，就是随机地遮盖掉一部分照片的像素。好比在照片上人为添加一定量的白噪音，从而让网络模型无法发现过多的细节，把注意力停留在大的框架和结构上；同时，丢弃层在每次迭代时都会随机地给照片添加不同的白噪音：**掩码**（mask），从而避免引入新的规律。

```{figure} images/dropout.png
:align: center
:width: 480px
**图例：丢弃层示意图**
```

In [1]:
from abc import abstractmethod, ABC
import numpy as np

np.random.seed(99)

## 基础架构

### 张量

In [2]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = np.zeros_like(self.data)
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.backward()

    @property
    def size(self):
        return np.prod(self.data.shape[1:])

    def __repr__(self):
        return f'Tensor({self.data})'

### 基础数据集

In [3]:
class Dataset(ABC):

    def __init__(self, batch_size=1):
        self.batch_size = batch_size

        self.test_labels = self.test_features = None
        self.train_labels = self.train_features = None

        self.load()
        self.train()

    @abstractmethod
    def load(self):
        pass

    def train(self):
        self.features = self.train_features
        self.labels = self.train_labels

    def eval(self):
        self.features = self.test_features
        self.labels = self.test_labels

    def shape(self):
        return Tensor(self.features).size, Tensor(self.labels).size

    def items(self):
        return Tensor(self.features), Tensor(self.labels)

    def __len__(self):
        return len(self.features) // self.batch_size

    def __getitem__(self, index):
        start = index * self.batch_size
        end = start + self.batch_size

        feature = Tensor(self.features[start: end])
        label = Tensor(self.labels[start: end])
        return feature, label

    def estimate(self, predictions):
        pass

### 基础层

在基础层，我们添加两个新的函数：

* **训练函数**（train）：切换到模型训练模式；
* **测试函数**（eval）：切换到模型测试模式。

一些层，可以依此判断进行不同的操作。比如丢弃层，只在训练模式下有效；在测试模式下，我们无需通过丢弃层控制训练效果。

In [4]:
class Layer(ABC):

    def __init__(self):
        self.training = True

    def __call__(self, x: Tensor):
        return self.forward(x)

    def train(self):
        self.training = True

    def eval(self):
        self.training = False

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    @property
    def parameters(self):
        return []

    def __repr__(self):
        return ''

### 基础损失函数

In [5]:
class Loss(ABC):

    def __call__(self, p: Tensor, y: Tensor):
        return self.loss(p, y)

    @abstractmethod
    def loss(self, p: Tensor, y: Tensor):
        pass

### 基础优化器

In [6]:
class Optimizer(ABC):

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    def reset(self):
        for p in self.parameters:
            p.grad = np.zeros_like(p.data)

    @abstractmethod
    def step(self):
        pass

### 基础模型

In [7]:
class Model(ABC):

    def __init__(self, layer, loss_fn, optimizer):
        self.layer = layer
        self.loss_fn = loss_fn
        self.optimizer = optimizer

    @abstractmethod
    def train(self, dataset, epochs):
        pass

    @abstractmethod
    def test(self, dataset):
        pass

## 数据

### MNIST 数据集

In [8]:
class MNISTDataset(Dataset):

    def __init__(self, filename, batch_size=1):
        self.filename = filename
        super().__init__(batch_size)

    def load(self):
        with (np.load(self.filename, allow_pickle=True) as f):
            self.train_features, self.train_labels = self.normalize(f['x_train'], f['y_train'])
            self.test_features, self.test_labels = self.normalize(f['x_test'], f['y_test'])

    @staticmethod
    def normalize(x, y):
        inputs = x / 255
        inputs = np.expand_dims(inputs, axis=1)
        targets = np.zeros((len(y), 10))
        targets[range(len(y)), y] = 1
        return inputs, targets

    def estimate(self, predictions):
        count = (predictions.data.argmax(axis=1) == self.labels.argmax(axis=1)).sum()
        total = len(self.labels)
        return count / total

## 模型

### 线性层

In [9]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        super().__init__()
        self.weight = Tensor(np.random.rand(out_size, in_size) / in_size)
        self.bias = Tensor(np.random.rand(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad += p.grad.T @ x.data
            self.bias.grad += np.sum(p.grad, axis=0)
            x.grad += p.grad @ self.weight.data

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    @property
    def parameters(self):
        return [self.weight, self.bias]

    def __repr__(self):
        return f'Linear[weight{self.weight.data.shape}; bias{self.bias.data.shape}]'

### 顺序层

在顺序层，我们需要在**训练函数**（train）和**测试函数**（eval）里同时调整所有子层的模式。

In [10]:
class Sequential(Layer):

    def __init__(self, layers):
        super().__init__()
        self.layers = layers

    def train(self):
        for l in self.layers:
            l.train()

    def eval(self):
        for l in self.layers:
            l.eval()

    def forward(self, x: Tensor):
        for l in self.layers:
            x = l(x)
        return x

    @property
    def parameters(self):
        return [p for l in self.layers for p in l.parameters]

    def __repr__(self):
        return '\n'.join(str(l) for l in self.layers if str(l))

### 展平层

In [11]:
class Flatten(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.array(x.data.reshape(x.data.shape[0], -1)))

        def gradient_fn():
            x.grad += p.grad.reshape(x.data.shape)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __repr__(self):
        return f'Flatten[]'

### 丢弃层

**前向传播**：丢弃层的输出数据是一个张量。在测试模式下，丢弃层直接将输入数据当作输出数据返回，不做任何操作。

在模型训练时，每次迭代，我们利用 NumPy 的 random 函数生成一个与输入数据结构相同的**随机掩码**。然后用掩码遮盖掉一部分输入值，将剩余的输入值传递给下一层。

**梯度计算**：在**梯度计算函数**（gradient_fn）中，我们必须使用**同一份掩码**来遮盖后一层传回來的梯度。只有在前向传播中参与了计算的位置，其对应的梯度才能向前一层继续传递。这样可以确保参数更新的逻辑与前向传播完全一致，避免了无效的权重调整。

同时我们把输入值加入**父节点列表**（parents）。

**反向传播**：展平层没有参数，所以**参数列表**（parameters）为空。

---

创建丢弃层，我们需要一个参数：**丢弃率**（Dropout Rate）,用于决定多少比例的输入数据将被遮盖掉。丢弃率可以看作一个新的**超参数**，我们将使用缺省值来简化模型。

In [12]:
class Dropout(Layer):

    def __init__(self, dropout_rate=0.2):
        super().__init__()
        self.dropout_rate = dropout_rate

    def forward(self, x: Tensor):
        if not self.training:
            return x

        mask = np.random.random(x.data.shape) > self.dropout_rate
        p = Tensor(x.data * mask)

        def gradient_fn():
            x.grad += p.grad * mask

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __repr__(self):
        return f'Dropout[rate={self.dropout_rate}]'

### ReLU 激活函数

In [13]:
class ReLU(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.maximum(0, x.data))

        def gradient_fn():
            x.grad += p.grad * (p.data > 0) 

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __repr__(self):
        return f'ReLU[]'

### 损失函数（均方误差）

In [14]:
class MSELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        mse = Tensor(np.mean(np.square(y.data - p.data)))

        def gradient_fn():
            p.grad += -2 * (y.data - p.data) / len(y.data)

        mse.gradient_fn = gradient_fn
        mse.parents = {p}
        return mse

### 优化器（随机梯度下降）

In [15]:
class SGDOptimizer(Optimizer):

    def step(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

### 神经网络模型

在模型中，我们不仅需要通过**训练函数**和**测试函数**调整**数据集**的模式，也需要以同样的方式调整**层**的模式。

In [16]:
class NNModel(Model):

    def train(self, dataset, epochs):
        self.layer.train()
        dataset.train()

        for epoch in range(epochs):
            for i in range(len(dataset)):
                features, labels = dataset[i]

                predictions = self.layer(features)
                loss = self.loss_fn(predictions, labels)
                self.optimizer.reset()
                loss.backward()
                self.optimizer.step()

    def test(self, dataset):
        self.layer.eval()
        dataset.eval()

        features, labels = dataset.items()
        predictions = self.layer(features)
        loss = self.loss_fn(predictions, labels)
        return predictions, loss

## 设置

### 学习率

In [17]:
LEARNING_RATE = 0.01

### 批大小

In [18]:
BATCH_SIZE = 2

### 轮次

In [19]:
EPOCHS = 10

## 训练

### 迭代

丢弃层的位置通常应该放在全连接层之后，参数比较多的位置。我们把它放在线性层的后面，让我们看看模型训练的效果有什么变化。

In [20]:
dataset = MNISTDataset('tinymnist.npz', BATCH_SIZE)
layer = Sequential([Flatten(),
                    Linear(dataset.shape()[0], 64),
                    ReLU(),
                    Dropout(),
                    Linear(64, dataset.shape()[1])])
loss_fn = MSELoss()
optimizer = SGDOptimizer(layer.parameters, lr=LEARNING_RATE)

model = NNModel(layer, loss_fn, optimizer)
model.train(dataset, EPOCHS)
print(layer)

Flatten[]
Linear[weight(64, 784); bias(64,)]
ReLU[]
Dropout[rate=0.2]
Linear[weight(10, 64); bias(10,)]


## 验证

### 测试

In [21]:
predictions, loss = model.test(dataset)
accuracy = dataset.estimate(predictions)
print(f'accuracy: {accuracy:.2%}')

accuracy: 90.90%


模型测试的准确率有所下降。丢弃层有助于提高模型的泛化能力，但代价是训练可能需要更多的轮次。

另一方面，MNIST 数据集的数据比较“干净”，所以丢弃层的作用不明显。

## 课后练习

尝试添加丢弃率作为超参数。观察调节丢弃率对训练效果的影响。